In [1]:
from itertools import product

## Functions

I. Tree class

In [2]:
def print_tab(n):
    for _ in range(3*n-2):
        print(' ', end='')
    print('`-', end='')

class Tree():
    def __init__(self, a, b, c=None):
        self.name = a
        self.left = b
        self.right = c
 
    def Print(self, level):
        print(self.name, end='--')
        level+=1
        if type(self.left)==Tree:
            self.left.Print(level)
        elif self.left!=None:
            print(self.left)

        if type(self.right)==Tree:
            print_tab(level)
            self.right.Print(level)
        elif self.right!=None:
            print_tab(level)
            print(self.right)
        level-=1

II. Extras

In [3]:
def cartesian_product(sets):
    return [''.join(element) for element in product(sets[0],sets[1])]

c = cartesian_product

def extract(li, lo):
    retv = []
    for i in li:
        for j in lo:
            retv.append([i,j])
    return retv

III. Preimage Finding

In [4]:
def preimage_word(word):
    retv = []
    for key in laws:
        retv += [[],[key]][word in laws[key]]
    return retv

def preimage_set(li):
    retv = []
    for word in li:
        retv += preimage_word(word)
    return list(set(retv))

def pc(sets):
    return preimage_set(c(sets))

IV. CYK table

In [5]:
def raw_row(betu):
    v = [[(table[i][start],table[betu-2-i][start+i+1]) for i in range(betu-1)] for start in range(len(text)+1-betu)]
    return [[pc(pair) for pair in pairs] for pairs in v]

def new_row(betu):
    row = []
    temp = raw_row(betu)
    for sets in temp:
        collector = []
        for li in sets:
            collector += li
        row.append(list(set(collector)))
    table[betu-1] = row

V. Tree from table

In [9]:
def children(non_terminal, row, column):
    retv = []
    for i in range(row):
        v = c([table[i][column],table[row-i-1][column+i+1]])
        for j in v:
            if j in laws[non_terminal]:
                retv.append([[j[0], i, column], [j[1], row-i-1, column+i+1]])
    return retv

def tree(non_terminal, row, column):
    if row == 0:
        return [Tree(non_terminal, text[column])]
    trees = []
    li = children(non_terminal, row, column)
    for i in li:
        for j in extract(tree(i[0][0],i[0][1],i[0][2]), tree(i[1][0],i[1][1],i[1][2])):
            trees.append(Tree(non_terminal,j[0],j[1]))
    return trees

VI. Tree from Tree

In [11]:
def tree_to_tree(node):
    if node.name in 'coam':
        return [node.left,node.right][node.name in 'co']

    if node.name=='I':
        return Tree('.',tree_to_tree(node.right))

    if node.name in 'SP':
        if node.right==None:
            return node.left
        else:
            retv = tree_to_tree(node.left)
            retv.right = tree_to_tree(node.right)
            
    if node.name in 'qr':
        if node.right.name in 'am':
            return Tree(node.right.left, tree_to_tree(node.left))
        else:
            retv = tree_to_tree(node.left)
            retv.right = tree_to_tree(node.right)
            
    return retv

## Run

input Rules

In [6]:
laws = {}
file = open('aritmethic', 'r')
for line in file:
    l = line[:-1].split('->')
    laws[l[0]] = l[1].split('|')
file.close()

the CYK

In [7]:
text = '(1+2)*(3+4)+1+3'
table = [[] for _ in text]

table[0] = [preimage_word(word) for word in text]
for number_of_letters in range(2,len(text)+1):
    new_row(number_of_letters)

In [8]:
for i in table[::-1]:
    print(i)

[['S']]
[['q'], []]
[['S'], [], []]
[['q'], [], [], []]
[['S', 'P'], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], [], ['S']]
[[], [], [], [], [], [], ['q'], []]
[[], [], [], [], [], [], ['S'], [], []]
[['r'], [], [], [], [], [], ['q'], [], [], []]
[['S', 'P'], [], [], [], [], [], ['S', 'P'], [], [], [], []]
[['I'], [], [], [], [], [], ['I'], [], [], [], [], []]
[[], ['S'], [], [], [], [], [], ['S'], [], [], [], [], ['S']]
[['I'], ['q'], [], [], [], [], ['I'], ['q'], [], [], [], [], ['q'], []]
[['o'], ['S', 'P'], ['a'], ['S', 'P'], ['c'], ['m'], ['o'], ['S', 'P'], ['a'], ['S', 'P'], ['c'], ['a'], ['S', 'P'], ['a'], ['S', 'P']]


tree Finding

In [10]:
a = tree('S',len(text)-1,0)[0]
a.Print(0)

S--q--S--r--P--I--o--(
                `-S--q--S--1
                      `-a--+
                   `-S--2
             `-c--)
          `-m--*
       `-P--I--o--(
             `-S--q--S--3
                   `-a--+
                `-S--4
          `-c--)
    `-a--+
 `-S--q--S--1
       `-a--+
    `-S--3


Tree to Tree

In [12]:
b = tree_to_tree(a)
b.Print(0)

+--*--.--+--1
          `-2
    `-.--+--3
          `-4
 `-+--1
    `-3
